Import Libraries

In [1]:
from shapely.geometry import shape          #AOI
import pystac_client                        #STAC API
from datetime import datetime, timedelta    #Time Range
import planetary_computer                   #Cloud-based data access
import rioxarray as rxr                     #loading raster data & reprojecting
import xarray as xr                         #Intergrate dataset into datacube
import matplotlib.pyplot as plt             #Visualization
import dask                                 #Lazy Loading
import cdsapi
import geopandas as gpd   #loading vector data
import rasterio as rio
import numpy as np


import json
import requests
import os

Defining the AOI

In [2]:
# Given AOI
aoi_geojson = {
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "properties": {},
            "geometry": {
                "coordinates": [[
                    [36.306872504019424, -0.5952725839718056],
                    [36.306872504019424, -0.7423752404067301],
                    [36.45206844566468, -0.7423752404067301],
                    [36.45206844566468, -0.5952725839718056],
                    [36.306872504019424, -0.5952725839718056]
                ]],
                "type": "Polygon"
            }
        }
    ]
}

# Convert to Bounding Box (min_lon, min_lat, max_lon, max_lat)
aoi_geometry = shape(aoi_geojson["features"][0]["geometry"])
min_lon, min_lat, max_lon, max_lat = aoi_geometry.bounds
print(f"AOI Bounding Box: {min_lon}, {min_lat}, {max_lon}, {max_lat}")


AOI Bounding Box: 36.306872504019424, -0.7423752404067301, 36.45206844566468, -0.5952725839718056


Connect to Stac API & Print all collections, specify Time Range

In [3]:
# Connect to a public STAC API 
STAC_API_URLS = [
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    "https://earth-search.aws.element84.com/v1"
]

os.environ["AWS_NO_SIGN_REQUEST"] = "YES"

# Initialize STAC client
client = pystac_client.Client.open(STAC_API_URLS[1])

# Time Range: Last 3 months
end_date = datetime.now()
start_date = end_date - timedelta(days=90)
date_range = f"{start_date.date()}/{end_date.date()}"

print(f"Searching data from {start_date.date()} to {end_date.date()}")

# Print available collections
#collections = client.get_all_collections()
#print("Available Collections:", [c.id for c in collections])


Searching data from 2024-11-30 to 2025-02-28


Query the STAC API for the Data

1. Query Sentinel-1 (VV, VH - Radar Backscatter) data & Load the Data as raster arrays

In [4]:
# Query Sentinel-1 GRD for VV & VH (Radar)
s1_search = client.search(
    collections=["sentinel-1-grd"], bbox=[min_lon, min_lat, max_lon, max_lat], datetime=date_range
)

# Display results for Sentinel-1 GRD for VV & VH
s1_items = list(s1_search.items())
print(f"Found {len(s1_items)} Sentinel-1 items")

# Sign URLs for Direct Access
s1_items = [planetary_computer.sign(item).assets for item in s1_items]

# Extract VV & VH Polarization URLs
vv_links = [item["vv"].href for item in s1_items if "vv" in item]
vh_links = [item["vh"].href for item in s1_items if "vh" in item]

print(f"Total VV Images: {len(vv_links)}")
print(f"Total VH Images: {len(vh_links)}")

# Load VV Polarization as Raster
vv_rasters = [rxr.open_rasterio(vv, masked=True) for vv in vv_links]
vh_rasters = [rxr.open_rasterio(vh, masked=True) for vh in vh_links]

print("All VV & VH rasters loaded successfully.")

# Plot Sample (First VV & VH Image)  for #testing



Found 7 Sentinel-1 items
Total VV Images: 7
Total VH Images: 7
All VV & VH rasters loaded successfully.


2. Query Sentinel-2 (NDVI - Vegetation Health) data & Load the Data as raster arrays

In [5]:
# Query Sentinel-2 L2A Data
s2_search = client.search(
    collections=["sentinel-2-l2a"], bbox=[min_lon, min_lat, max_lon, max_lat], datetime=date_range
)

# Extract Items
s2_items = list(s2_search.items())
print(f"Found {len(s2_items)} Sentinel-2 L2A items")

# Ensure we have at least one item
if not s2_items:
    raise ValueError("No Sentinel-2 L2A data found for the given AOI and date range.")

# Authenticate all items for cloud access
s2_items = [planetary_computer.sign(item) for item in s2_items]


# Load all available B04 (Red) and B08 (NIR) bands
# Open bands with chunking for lazy loading
B4_rasters = [
    rxr.open_rasterio(item.assets["red"].href, masked=True, chunks={'x':1024, 'y':1024}).squeeze()
    for item in s2_items
]
B8_rasters = [
    rxr.open_rasterio(item.assets["nir"].href, masked=True, chunks={'x':1024, 'y':1024}).squeeze()
    for item in s2_items
]

# Compute NDVI lazily
NDVI_rasters = [ (nir - red) / (nir + red) for red, nir in zip(B4_rasters, B8_rasters) ]

print("All NDVI rasters loaded (lazily) successfully.")
# Plot a sample NDVI image (First available) for #testing



Found 32 Sentinel-2 L2A items
All NDVI rasters loaded (lazily) successfully.


3. Query (Temperature - Climatic Variations) data & Load the Data as raster arrays

In [6]:


# Define bounding box and time range
# ERA5 expects [North, West, South, East]
bbox = [-0.5952725839718056, 36.306872504019424, -0.7423752404067301, 36.45206844566468] 
start_date = "2024-12-01"  # December 1, 2024
end_date = "2025-02-28"    # February 28, 2025
date_range = f"{start_date}/{end_date}"

# Initialize the CDS API client
c = cdsapi.Client()

# Download ERA5 daily 2m temperature data
c.retrieve(
    "reanalysis-era5-single-levels",
    {
        "variable": "2m_temperature",
        "product_type": "reanalysis",
        "year": ["2024", "2025"],  
        "month": ["12", "01", "02"],  # Last 3 monthsa
        "day": [str(i).zfill(2) for i in range(1, 32)],  # All days
        "time": "12:00",
        "format": "netcdf",
        "area": bbox,  # Full AOI region
    },
    "era5_temperature.nc"
)



2025-02-28 20:28:23,904 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-02-28 20:28:23,905 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.
2025-02-28 20:28:24,781 INFO Request ID is 3f026a15-9c6b-455c-b0f1-7fb6c7ba5446
2025-02-28 20:28:24,995 INFO status has been updated to accepted
2025-02-28 20:28:33,952 INFO status has been updated to running
2025-02-28 20:29:00,558 INFO status has been updated to successful


5128b8e2eb150f61cfdeb693afc2732a.nc:   0%|          | 0.00/31.4k [00:00<?, ?B/s]

'era5_temperature.nc'

In [8]:
# Load the NetCDF file using xarray
# Use the netCDF4 engine explicitly
era5_data = xr.open_dataset("era5_temperature.nc", engine="netcdf4")

# Assuming 'temperature_raster' is your xarray.DataArray
temperature_raster = era5_data["t2m"] - 273.15  # Convert to Celsius

# Define the affine transform for your raster data
# Here we use the origin (longitude, latitude) and the pixel size (spacing between pixels).
# Adjust the parameters based on your data’s actual spatial characteristics.
transform = rio.transform.from_origin(west=36.31, north=-0.743, xsize=0.5, ysize=0.5)

# Now you can write the raster to a GeoTIFF with the correct georeferencing information
temperature_raster.rio.write_transform(transform, inplace=True)

# Save the data as a GeoTIFF file
temperature_raster.rio.to_raster("temperature_raster.tif")

# Open the raster using rioxarray
temperature_raster = rxr.open_rasterio("temperature_raster.tif")

# Print metadata and shape
print("========== ERA5 Temperature Raster ==========")
print(f"Date Range: {date_range}")
print("Raster Shape (time, lat, lon):", temperature_raster.shape)

print(temperature_raster)

========== ERA5 Temperature Raster ==========
Date Range: 2024-12-01/2025-02-28
Raster Shape (time, lat, lon): (145, 1, 1)
<xarray.DataArray (band: 145, y: 1, x: 1)> Size: 580B
[145 values with dtype=float32]
Coordinates:
  * band         (band) int64 1kB 1 2 3 4 5 6 7 ... 139 140 141 142 143 144 145
  * x            (x) float64 8B 36.31
  * y            (y) float64 8B -0.743
    spatial_ref  int64 8B 0
Attributes:
    scale_factor:  1.0
    add_offset:    0.0
    long_name:     t2m


4. Query (Elevation - Terrain Impact)data & Load the Data as raster arrays

In [9]:
# OpenTopography API Endpoint
opentopo_url = "https://portal.opentopography.org/API/globaldem"

# Define AOI Bounding Box
params = {
    "demtype": "SRTMGL1",  # SRTMGL1, COP30, or COP90
    "west": 36.306872,      # Min Longitude
    "south": -0.742375,     # Min Latitude
    "east": 36.452068,      # Max Longitude
    "north": -0.595272,     # Max Latitude
    "outputFormat": "GTiff",  # GeoTIFF format
    "API_Key": "5a856dfd43a0f226322ccd13bb7b9654"  # Load API key from environment variable
}

# Make API request
response = requests.get(opentopo_url, params=params)

# Check if request was successful
if response.status_code == 200:
    # Save the GeoTIFF file
    dem_path = "elevation.tif"
    with open(dem_path, "wb") as file:
        file.write(response.content)
    print(f"Elevation data saved as {dem_path}")
else:
    raise Exception(f"Failed to fetch elevation data: {response.text}")

# Load the GeoTIFF as a raster array
elevation_raster = rxr.open_rasterio(dem_path, masked=True)
elevation_raster.name = "Elevation"

# Print metadata
print(elevation_raster)


Elevation data saved as elevation.tif
<xarray.DataArray 'Elevation' (band: 1, y: 530, x: 523)> Size: 1MB
[277190 values with dtype=float32]
Coordinates:
  * band         (band) int64 8B 1
  * x            (x) float64 4kB 36.31 36.31 36.31 36.31 ... 36.45 36.45 36.45
  * y            (y) float64 4kB -0.5953 -0.5956 -0.5958 ... -0.7419 -0.7422
    spatial_ref  int64 8B 0
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0


Reproject the Data to a Common CRS & Resolution

In [ ]:

# Define target CRS
target_crs = "EPSG:4326"

# Ensure CRS is set correctly before reprojecting
for vv in vv_rasters:
    vv.rio.write_crs("EPSG:4326", inplace=True)  # Adjust if different

for vh in vh_rasters:
    vh.rio.write_crs("EPSG:4326", inplace=True)

# Reproject using nearest resampling & chunking
vv_reprojected = [vv.rio.reproject(target_crs, resampling=rio.enums.Resampling.nearest, blockxsize=1024, blockysize=1024) for vv in vv_rasters]
vh_reprojected = [vh.rio.reproject(target_crs, resampling=rio.enums.Resampling.nearest, blockxsize=1024, blockysize=1024) for vh in vh_rasters]

print("✅ Reprojection completed for VV & VH rasters.")


Integrate the datasets into a Datacube & save it into a portable format

Inspecting the Datacube